# Simulate a feedback survey
This notebook provides sample EDSL code for simulating surveys with AI agents and large language models.
In the steps below we use EDSL to prompt LLMs to suggest names for a yoga studio, and then simulate a feedback survey with AI agents representing target customers.

Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Create a question
We start by creating a question to prompt a language model to suggest some names for a yoga studio.
EDSL comes with many common [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model - multiple choice, free text, linear scale, etc. 
Here we use `QuestionList` to prompt a model to return a list of items:

In [1]:
from edsl import QuestionList

In [2]:
q = QuestionList(
    question_name = "yoga_studio_name",
    question_text = "What are some creative names for a yoga studio?",
    max_list_items = 10
)

## Select some models to answer the question
EDSL works with many popular [language models that we can select](https://docs.expectedparrot.com/en/latest/language_models.html) to generate responses to questions.

In [3]:
from edsl import ModelList, Model

To see a list of all services:

In [4]:
Model.services()

['openai',
 'anthropic',
 'deep_infra',
 'google',
 'groq',
 'bedrock',
 'azure',
 'ollama',
 'test',
 'mistral',
 'together']

To see a list of all available models (uncomment and run the following code):

In [5]:
# Model.available() 

To select models to use with a question or survey:

In [6]:
models = ModelList(
    Model(m) for m in ["gemini-1.5-flash", "gpt-4o"]
)

## Run the question 
We administer a question to a language model by calling the `run()` method on it.
This generates a formatted dataset of `Results`:

In [7]:
results = q.by(models).run()

## Inspect the results
EDSL comes with built-in [methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html).
Here we inspect the responses for each model:

In [8]:
results.select("model", "yoga_studio_name").print(format="rich")

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ model            ┃ answer                                                                                       ┃
┃ .model           ┃ .yoga_studio_name                                                                            ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt-4o           │ ['Zen Flow', 'Soul Stretch', 'Harmony Haven', 'Tranquil Twist', 'Serene Space', 'Blissful    │
│                  │ Balance', 'Mindful Movements', 'Peaceful Pose', 'Inner Light Studio', 'Namaste Nook']        │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ gemini-1.5-flash │ ['The Flowing Lotus', 'Inner Peace Sanctuary', 'Sun & Moon Yoga', 'Mindful Movement Studio', │
│                  │ 'The Yoga Tree', 'Namaste Haven', 'Zenith Yoga', 'The Balanced Body', 'Asana & Aura',        │
│                  │ 'Harmony Studio']                                                                            │
└──────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

## Use the responses in new questions
We can format the responses to use them as options to new questions:

In [9]:
gemini_names = results.filter("model.model == 'gemini-1.5-flash'").select("yoga_studio_name").to_list()[0]
gemini_names

['The Flowing Lotus',
 'Inner Peace Sanctuary',
 'Sun & Moon Yoga',
 'Mindful Movement Studio',
 'The Yoga Tree',
 'Namaste Haven',
 'Zenith Yoga',
 'The Balanced Body',
 'Asana & Aura',
 'Harmony Studio']

In [10]:
gpt4o_names = results.filter("model.model == 'gpt-4o'").select("yoga_studio_name").to_list()[0]
gpt4o_names

['Zen Flow',
 'Soul Stretch',
 'Harmony Haven',
 'Tranquil Twist',
 'Serene Space',
 'Blissful Balance',
 'Mindful Movements',
 'Peaceful Pose',
 'Inner Light Studio',
 'Namaste Nook']

We can optionally randomize the list of options:

In [11]:
studio_names = gemini_names + gpt4o_names

In [12]:
import random

random.shuffle(studio_names)

## Constructing a survey
Here we create some new questions in different types and combine them into a survey to administer them together:

In [13]:
from edsl import QuestionMultipleChoice, QuestionCheckBox, QuestionRank, QuestionTopK, Survey

In [14]:
q1 = QuestionMultipleChoice(
    question_name = "favorite",
    question_text = "What is your favorite name for a yoga studio?",
    question_options = studio_names
)

In [15]:
q2 = QuestionCheckBox(
    question_name = "pick",
    question_text = "Pick the best names for a yoga studio.",
    question_options = studio_names,
    min_selections = 2,
    max_selections = 4
)

In [16]:
q3 = QuestionRank(
    question_name = "rank",
    question_text = "Rank your favorite names for a yoga studio.",
    question_options = studio_names,
    num_selections = 5
)

In [17]:
survey = Survey([q1, q2, q3])

## Designing agents to answer the survey
Next we can design AI agents with relevant traits to answer the questions.
Here we use a model to draft some personas, and then create "agent" objects for them to use with the survey:

In [18]:
q = QuestionList(
    question_name = "personas",
    question_text = "Draft 5 diverse personas for patrons of a yoga studio."
)

In [19]:
personas = q.run().select("personas").to_list()[0]
personas

['A 28-year-old tech professional seeking stress relief and mindfulness after long work hours',
 'A 45-year-old mother of three looking to improve flexibility and find personal time',
 'A 60-year-old retired teacher using yoga to maintain health and social connections',
 'A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention',
 'A 35-year-old artist exploring yoga for creative inspiration and community involvement']

In [20]:
from edsl import AgentList, Agent

In [21]:
agents = AgentList(
    Agent(traits = {"persona":p}) for p in personas
)

### Run the survey with the agents
Here we add the agents to the survey and run it with the language models we selected above (to compare responses):

In [22]:
results = survey.by(agents).by(models).run()

In [23]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "favorite", "pick", "rank")
    .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ model            ┃ agent                  ┃ answer            ┃ answer                 ┃ answer                 ┃
┃ .model           ┃ .persona               ┃ .favorite         ┃ .pick                  ┃ .rank                  ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemini-1.5-flash │ A 22-year-old college  │ The Flowing Lotus │ ['The Balanced Body',  │ ['Sun & Moon Yoga',    │
│                  │ athlete incorporating  │                   │ 'The Yoga Tree', 'Sun  │ 'Asana & Aura',        │
│                  │ yoga for enhanced      │                   │ & Moon Yoga', 'Zenith  │ 'Zenith Yoga',         │
│                  │ performance and injury │                   │ Yoga']                 │ 'Namaste Haven', 'Zen  │
│                  │ prevention             │                   │                        │ Flow']                 │
├──────────────────┼────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ gemini-1.5-flash │ A 28-year-old tech     │ Harmony Haven     │ ['Serene Space',       │ ['Mindful Movements',  │
│                  │ professional seeking   │                   │ 'Mindful Movements',   │ 'Blissful Balance',    │
│                  │ stress relief and      │                   │ 'Harmony Haven',       │ 'Harmony Studio', 'Sun │
│                  │ mindfulness after long │                   │ 'Inner Peace           │ & Moon Yoga', 'Zen     │
│                  │ work hours             │                   │ Sanctuary']            │ Flow']                 │
├──────────────────┼────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ gemini-1.5-flash │ A 35-year-old artist   │ The Flowing Lotus │ ['Blissful Balance',   │ ['Sun & Moon Yoga',    │
│                  │ exploring yoga for     │                   │ 'The Yoga Tree', 'Sun  │ 'Inner Light Studio',  │
│                  │ creative inspiration   │                   │ & Moon Yoga', 'Namaste │ 'Namaste Haven',       │
│                  │ and community          │                   │ Nook']                 │ 'Inner Peace           │
│                  │ involvement            │                   │                        │ Sanctuary', 'The       │
│                  │                        │                   │                        │ Flowing Lotus']        │
├──────────────────┼────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ gemini-1.5-flash │ A 45-year-old mother   │ Harmony Haven     │ ['The Balanced Body',  │ ['Mindful Movements',  │
│                  │ of three looking to    │                   │ 'The Yoga Tree',       │ 'Blissful Balance',    │
│                  │ improve flexibility    │                   │ 'Harmony Haven',       │ 'The Yoga Tree', 'The  │
│                  │ and find personal time │                   │ 'Inner Peace           │ Flowing Lotus',        │
│                  │                        │                   │ Sanctuary']            │ 'Namaste Nook']        │
├──────────────────┼────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ gemini-1.5-flash │ A 60-year-old retired  │ The Yoga Tree     │ ['Blissful Balance',   │ ['Mindful Movements',  │
│                  │ teacher using yoga to  │                   │ 'The Yoga Tree', 'Sun  │ 'Blissful Balance',    │
│                  │ maintain health and    │                   │ & Moon Yoga', 'Namaste │ 'The Yoga Tree', 'Sun  │
│                  │ social connections     │                   │ Nook']                 │ & Moon Yoga', 'Namaste │
│                  │                        │                   │                        │ Haven']                │
├──────────────────┼────────────────────────┼───────────

We could also solicit feedback on individual names.
This can be done by creating a "scenario" of each question for each name:

In [24]:
from edsl import QuestionLinearScale, QuestionFreeText, Survey, ScenarioList

In [25]:
s = ScenarioList.from_list("studio_name", studio_names)

In [26]:
q_memorable = QuestionLinearScale(
    question_name = "memorable",
    question_text = "How memorable is this yoga studio name: {{ studio_name }}",
    question_options = [1,2,3,4,5],
    option_labels = {1:"Not at all memorable", 5:"Very memorable"}
)

In [27]:
q_negative = QuestionFreeText(
    question_name = "negative",
    question_text = "What are some negative things people might say about this yoga studio name: {{ studio_name }}"
)

In [28]:
results = Survey([q_memorable, q_negative]).by(s).by(agents).by(models).run()

In [30]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "studio_name", "memorable", "negative")
    .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ model            ┃ agent                     ┃ scenario                ┃ answer     ┃ answer                    ┃
┃ .model           ┃ .persona                  ┃ .studio_name            ┃ .memorable ┃ .negative                 ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemini-1.5-flash │ A 22-year-old college     │ Sun & Moon Yoga         │ 4          │ Hmm, "Sun & Moon Yoga"?   │
│                  │ athlete incorporating     │                         │            │ It's kind of... generic,  │
│                  │ yoga for enhanced         │                         │            │ you know? Like, it's not  │
│                  │ performance and injury    │                         │            │ bad, but it doesn't       │
│                  │ prevention                │                         │            │ really stand out.  I've   │
│                  │                           │                         │            │ heard some people say it  │
│                  │                           │                         │            │ sounds a bit too          │
│                  │                           │                         │            │ "hippie-dippie" for them. │
│                  │                           │                         │            │ You know, like it's all   │
│                  │                           │                         │            │ about crystals and        │
│                  │                           │                         │            │ incense and stuff.  And,  │
│                  │                           │                         │            │ honestly, that's not      │
│                  │                           │                         │            │ really what I'm looking   │
│                  │                           │                         │            │ for in a yoga studio.  I  │
│                  │                           │                         │            │ want something that's     │
│                  │                           │                         │            │ going to help me get      │
│                  │                           │                         │            │ stronger and more         │
│                  │                           │                         │            │ flexible, not just        │
│                  │                           │                         │            │ something that's all      │
│                  │                           │                         │            │ about "vibes."            │
├──────────────────┼───────────────────────────┼─────────────────────────┼────────────┼───────────────────────────┤
│ gemini-1.5-flash │ A 22-year-old college     │ Zenith Yoga             │ 4          │ Hmm, "Zenith Yoga" sounds │
│                  │ athlete incorporating     │                         │            │ pretty cool, right?  It's │
│                  │ yoga for enhanced         │                         │            │ got that "reaching your   │
│                  │ performance and injury    │                         │            │ peak" vibe, which is      │
│                  │ prevention                │                         │            │ totally what I'm going    │
│                  │                           │                         │            │ for with my training.     │
│                  │                           │                         │            │ But I gotta admit, it     │
│                  │                           │                         │            │ might sound a             │
│                  │                           │                         │            │ little...intense?  Like,  │
│                  │                           │        

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook to share with others (visibility can be *public*, *private* or *unlisted* by default):

In [31]:
from edsl import Notebook

In [32]:
n = Notebook(path = "yoga_studio_name_survey.ipynb")

In [33]:
n.push(description = "Feedback on names for a yoga studio", visibility = "public")

{'description': 'Feedback on names for a yoga studio',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/e21abb19-672e-4dc8-ab7f-4a31a6c21d53',
 'uuid': 'e21abb19-672e-4dc8-ab7f-4a31a6c21d53',
 'version': '0.1.33',
 'visibility': 'public'}

To update an object at the Coop:

In [36]:
n = Notebook(path = "yoga_studio_name_survey.ipynb")

In [37]:
n.patch(uuid = "e21abb19-672e-4dc8-ab7f-4a31a6c21d53", value = n)

{'status': 'success'}